In [5]:
import pandas as pd
import numpy as np
import re
import requests
from bs4 import BeautifulSoup
from time import sleep

In [2]:
head = {x[0]:x[1] for x in re.findall(r'(.*): (.*)',
'''Accept-Language: en-US,en;q=0.9
Connection: keep-alive
Cookie: queries=l%3DPhilippines%26lid%3D113800%26c%3Dinformation-technology-telecoms%26radius%3D0; __utma=40482480.720908387.1560410529.1560845769.1560907543.4; __utmc=40482480; __utmz=40482480.1560907543.4.4.utmcsr=google|utmccn=(organic)|utmcmd=organic|utmctr=(not%20provided); __utmt=1; so=0; as=0; su19hd7=c49a015df5d1991520cb3b8f8332cadf4c60e2465ebd19b29170c2c98511440c53bf33ba9b52bf0c18c6dfdbb79c4a55839bb58715337c381bce78482291dc94357c4a6554d3cdd69de9ec506932e8b65594d145260ebc073e534a441f91a11d8ba4a12f95aeeb56b1cfbd114edf88cd2ba17d0a665d56569818a1c420c099ccd38eec6bbeb49a51718bf867d72d0375d91f286471dea3a2e5f5556ec65475dfd97f37995db285b2c8cd8933c57d800b1d6739b81e4e50fa6cff81ffc4b5b5e4368daa57ad13e73febe1a89093706329b9281de4b3f9c73129dafdb8ff309fa88e71b691b5b32bf4312c5c6bc1443d9202ad8b999a0953fdcf5df9a66c86ebd06ea41e3701fa32c45318ef815878a7d749254949197e1e05df95b89e63282f7d8667c160481784e29dbff1d0eb4c0df0601a26e1feebd0213db098f79ead63c1f545094190f2432bc1698ab390e29b18051f5c7b328c530d7f7d832419935357ad1df7a22c900bedafeb6f3e7f533e4adaa1586115d2646a4fcf257afe81062e26eb5df178d188f4207cb3a9827ae0ea60b0af00e2f7d3578ddf3ab39bf20e7f8ee6b85aef5dcc2e493c25334f93ec28ead8a6c8e749af6cea62a00c6ad99c5236ff636ec15397741c0d50cbbea34144dc8b3cf3f447b6a57813617625f0a8885142b78e1d9a6389916e849d8a566a1a7a3a96d8bf9e1cd85b77a37a4949d49f892cecdbe9233beb3f65c43034987a249013a9a4cadefb559ed157ddfb61c22459b192d5e54409f0b8fb243d790a6341a411ebe3e65b6972afe08c34ee4a1048; sses=727539fabc62da99ed3a0227d3875d73aa1fa04384f97b214e73a73bb24bb8c24cc014d711223e84936ac2cd9002abc75963a87fcc221a67; __utmb=40482480.6.10.1560907543
Host: www.careerjet.ph
Upgrade-Insecure-Requests: 1
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36
''',re.M)}

In [6]:
sesh = requests.session()
sesh.headers.update(head)
t1 = 7

In [16]:
job_ads = []
for pg in np.arange(1,1981,20):
    url = f'https://www.careerjet.ph/wsearch/jobs?c=information-technology-telecoms&l=Philippines&lid=113800&b={pg}&cid=52'
    cjet_pg = sesh.get(url)
    cjet_sp = BeautifulSoup(cjet_pg.text, 'lxml')
    cj_jobs = cjet_sp.findAll('div',{'class':['job', 'display-new-job', 'clickable', 'first']})
    job_ads.append(cj_jobs)
    print(cjet_pg)
    sleep(t1)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200

In [91]:
# There's duplicate elements in the soup, delete duplicates via DataFrame!
job_det = []
for adlist in job_ads:
    for ad in adlist:
        
        # Get job details as a dictionary
        # Find only div elements with a title!
        if ad.findAll('a',{'class':'title-company'}) != []:
            tmp_dict = {'title':ad.findAll('a',{'class':'title-company'})[0].text,
                        'url':ad.findAll('a',{'class':'title-company'})[0].get('href')}
            # If Location is not specified then place in None
            if ad.findAll('a',{'class':'locations_compact'}) != []:
                tmp_dict.update({'location':ad.findAll('a',{'class':'locations_compact'})[0].text})
            else:
                tmp_dict.update({'location':'None'})
            # If Salary is not specified then place in None
            if ad.select('div.locations_compact.price') != []:
                tmp_dict.update({'salary':ad.select('div.locations_compact.price')[0].text})
            else:
                tmp_dict.update({'salary':'None'})
        # Append job details into a list
        job_det.append(tmp_dict)

df = pd.DataFrame(job_det).drop_duplicates().reset_index(drop=True)
df['url'] = df['url'].apply(lambda x: 'https://www.careerjet.ph'+x)
df.to_csv('Summaries/CareerJet/index.csv', index=False)

In [106]:
dftest = pd.read_csv('Summaries/CareerJet/index.csv')

head = {x[0]:x[1] for x in re.findall(r'(.*): (.*)',
'''Accept-Language: en-US,en;q=0.9
Cache-Control: max-age=0
Connection: keep-alive
Cookie: queries=l%3DPhilippines%26lid%3D113800%26c%3Dinformation-technology-telecoms%26radius%3D0; __utmc=40482480; __utmz=40482480.1560907543.4.4.utmcsr=google|utmccn=(organic)|utmcmd=organic|utmctr=(not%20provided); as=0; __utma=40482480.720908387.1560410529.1560920083.1560929945.7; __utmt=1; su19hd7=c64a47ca6ac402afbd1c98a6d7176045ca754756fc2aee2c64dbf25865ef0d166d9493512770c7539f164a58c285a0b829d03fe8034e7057fb13dacc006768b01acb046ec60acd5761a17b01339fcc09873a054c622d261eb776acf95ecc8deb27919c1a7ea22f699f83632048a9308a41cead49e65ea27a10c2ed980ef9d535569316d3ac382a1e0160596170799e5e3ef504ebc40cad5321fc12b22e8c19b8dd75d20ce0f9da1deb5cc23e15183011acc775bf958cfc2c7d4acbff62a510f4929bcb8e3ec50f19d4a42f5f62daa252f7c50d91a930f8fd7fb8f08a3e4bfda833914b5f9b45ee3a9e6ab5c639685d2f6183e0adaaaa0f1422263b8bebb42f2c2b0711bf73b1d326af6c39134b356b6cd0352bec875305e6fa4895012c52ef19f478542fe9b5fd03144eb0e24d8986618db26d82ec639772a3dda9d0eeeec0b96992e27bcc4ed0703a4be3d5a4112d6e0236b998664b5ea3f754507669813c31b166752a3cfb58b8ecf11c93bf47400b2bf11755f89fbf39065a46842ad40a5a75eeec0233cd2370507cb6a0dfeae0d982f063d49f0153c332bdd1411b4844ff15c5f5ec243a220a291837a2cf1e55753ba4f4e239b1f4e018017876d691bc900c18acb58cf9cb4136054fcf73bdbc1908b8614a556bd50a15a0aa102f85f64aadef6a1d81e6f861bcfffa447c173dd6655c460ac5cf018fd7081398ea250fd78b6530ccc9bbb9adb2b99f9e328c3e10f47b9e54468a9e9460abfd9bf4079c7956c9abee803b3e12c4032acd243500cbf6ff9c9b407a337e7ad2c8329a5a39a84c22c00d24eab6afc8226ae4905594aa; sses=4eef06bd37ab6ad2d8b7e097c03b71f3b5e01445a1162408c8ab956f6115ad5c5419b136415c95fd7d167e538655b7c9acb5cbd5c9ef0e86761a9138cc3454a6; so=0; __utmb=40482480.3.10.1560929945
Host: www.careerjet.ph
Upgrade-Insecure-Requests: 1
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36
''',re.M)}

In [112]:
job_desc = []
for link in dftest['url'].iloc[420:]:
    try:
        job_pg = sesh.get(link)
        print(job_pg)
        job_sp = BeautifulSoup(job_pg.text, 'lxml')
        desc = job_sp.findAll('div',{'class':'advertise_compact'})[0].text
        job_desc.append({'url':link, 'description':desc})
    except:
        print(f'skipping: {link}')
    sleep(t1)

<Response [200]>
<Response [404]>
skipping: https://www.careerjet.ph/job/52152167b38ec8ab7bb39c942996a3f0.html
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [404]>
skipping: https://www.careerjet.ph/job/17a0a6cb377e266c81876435023105a1.html
<Response [404]>
skipping: https://www.careerjet.ph/job/765fdc64b21c570684273449c2fc823a.html
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [404]>
skipping: https://www.careerjet.ph/job/38171cc69a726d5093bf00138c598b17.html
<Response [200]>
<Response [200]>
<Response [2

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [404]>
skipping: https://www.careerjet.ph/job/09934d5247dee9120ced3c94ba7e0d4d.html
<Response [200]>
<Response [404]>
skipping: https://www.careerjet.ph/job/e7748b67b96c3e190d7e7d11f8d3063e.html
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [20

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [404]>
skipping: https://www.careerjet.ph/job/7f6267938ff08fdee421e470b3aaf9b0.html
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Resp

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [404]>
skipping: https://www.careerjet.ph/job/b43c1ae4b056690d130882bb296bd95b.html
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
skipping: https://www.careerjet.ph/job/8b8d43c7555c9a176b3756e52b6b38f8.html
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [20

In [113]:
len(job_desc)

1504

In [114]:
pd.DataFrame(job_desc).to_csv(r'Summaries/CareerJet/2.csv', index=False)